# Generate Perfect Data to Classify with ID3
Perfect data is where everything seperates out and there is no noise, or contradictory datapoints.

I construct a decision tree in if loops to label data:

In [10]:
def decision(v, intervals):
    
    low_bound = intervals.pop(0)
    for i,upper_bound in enumerate(intervals):
        if low_bound<v<upper_bound:
            return i
        low_bound = upper_bound
    return -1

def f(x, y): # each variable is between 0 and 1
    
    match decision(x, [0, 0.5, 0.7, 1]):
        case 0:
            return 'A'
        case 1:
            match decision(y, [0, 0.5, 0.7, 1]):
                case 0:
                    return 'B'
                case 1:
                    return 'C'
                case 2:
                    return 'D'
        case 2:
            return 'E'


In [4]:
import numpy as np
import pandas as pd

# Generate Data

In [32]:
train = pd.DataFrame(
    [
        [decision(x, [0, 0.5, 0.7, 1]), decision(y, [0, 0.5, 0.7, 1]), f(x, y)]
        for x, y in np.random.uniform(size=[100,2])
    ],
    columns=['x', 'y', 'label']
)
train

,x,y,label
0,1,0,B
1,0,1,A
2,2,2,E
3,0,0,A
4,0,0,A
...,...,...,...
95,2,0,E
96,2,2,E
97,2,0,E
98,1,0,B


In [33]:
run ../ID3.py

In [34]:
model = ID3(train, 'label', attribute_values={
    'x': [0, 1, 2],
    'y': [0, 1, 2]
}, max_depth=7)

In [35]:
model

{'y=0': {'x=0': 'A', 'x=1': 'B', 'x=2': 'E'},
 'y=1': {'x=0': 'A', 'x=1': 'C', 'x=2': 'E'},
 'y=2': {'x=0': 'A', 'x=1': 'D', 'x=2': 'E'}}

In [37]:
errors = 0
def check_example(example, node):
    global errors
    if type(node)==str:
        # print(node, example['label'])
        assert(example['label']==node)
        if not example['label']==node:
            errors+=1
            return False
        return True
    
    keys = list(node.keys())
    
    attribute = keys[0].split('=')[0]
    
    value = example[attribute]
    
    # print(attribute+'='+str(value), example.label)
    return check_example(example, node[attribute+'='+str(value)])

for index, row in train.iterrows():
    # print(index)
    if not check_example(row, model):
        # print(index)
        pass
    
errors

0